In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]
RESULTS_DIR = 'results_003_lang_id_classification_2_weight_decay_2'
GPU_ID = 0

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
bias_category_specification = [
    {
        "category": "utterance",
        "subcategories": [
            {
                "subcategory": "verbal_nod",
                "tags": set(["utt-verbal-nod"])
            },
            {
                "subcategory": "multilingual",
                "tags": set(["utt-multi-lingual", "utt-multi-lingual-named-endity"])
            }
        ]
    },
    {
        "category": "speaker_count",
        "subcategories": [
            {
                "subcategory": "single",
                "tags": set(["spkr-single"])
            },
            {
                "subcategory": "multiple",
                "tags": set(["spkr-mult", "spkr-multi"])
            }
        ]
    },
    {
        "category": "gender",
        "subcategories": [
            {
                "subcategory": "male",
                "tags": set(["spkr-male"])
            },
            {
                "subcategory": "female",
                "tags": set(["spkr-female"])
            },
        ]
    },
    {
        "category": "language",
        "subcategories": [
            {
                "subcategory": "susu",
                "tags": set(["lng-susu"])
            },
            {
                "subcategory": "maninka",
                "tags": set(["lng-maninka"])
            },
            {
                "subcategory": "pular",
                "tags": set(["lng-pular"])
            }
        ]
    },
    {
        "category": "channel",
        "subcategories": [
            {
                "subcategory": "telephone",
                "tags": set(["ct-telephone"])
            },
            {
                "subcategory": "noise",
                "tags": set(["ct-noise"])
            },
            {
                "subcategory": "music",
                "tags": set(["ct-fg-music", "ct-tr-music", "ct-bg-music"])
            }
        ]
    }
]

flat_bias_category_specification = {}

for c in bias_category_specification:
    for sc in c['subcategories']:
        k = f"{c['category']}_{sc['subcategory']}"
        flat_bias_category_specification[k] = sc['tags']

_ = [print(f"{k}: {v}") for k,v in flat_bias_category_specification.items()]

utterance_verbal_nod: {'utt-verbal-nod'}
utterance_multilingual: {'utt-multi-lingual', 'utt-multi-lingual-named-endity'}
speaker_count_single: {'spkr-single'}
speaker_count_multiple: {'spkr-mult', 'spkr-multi'}
gender_male: {'spkr-male'}
gender_female: {'spkr-female'}
language_susu: {'lng-susu'}
language_maninka: {'lng-maninka'}
language_pular: {'lng-pular'}
channel_telephone: {'ct-telephone'}
channel_noise: {'ct-noise'}
channel_music: {'ct-tr-music', 'ct-bg-music', 'ct-fg-music'}


In [5]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [6]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label, tag_set
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels, audio_tags = zip(*data)

## Inspect label counts

In [7]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [8]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels, audio_tags = zip(*balanced_data)

In [9]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


## Inspect bias category counts in balanded data

In [10]:
def inspect_bias_category_counts():
    for name, tags in flat_bias_category_specification.items():
        count = len([ts for ts in audio_tags if len(tags.intersection(ts))>0])
        print(name, count, "/", len(audio_tags))
            
inspect_bias_category_counts()

utterance_verbal_nod 48 / 84
utterance_multilingual 21 / 84
speaker_count_single 25 / 84
speaker_count_multiple 58 / 84
gender_male 81 / 84
gender_female 15 / 84
language_susu 28 / 84
language_maninka 28 / 84
language_pular 28 / 84
channel_telephone 27 / 84
channel_noise 21 / 84
channel_music 24 / 84


# Prepare 10 cross validation folds

In [11]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [12]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [13]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [14]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [15]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [16]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [17]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [19]:
class LangIdLogisticReg1(nn.Module):
    def __init__(self):
        super(LangIdLogisticReg1, self).__init__()
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        x = self.lin1(x)
        return x
    
    
class LangIdDNN1(nn.Module):
    def __init__(self):
        super(LangIdDNN1, self).__init__()
        self.lin1 = nn.Linear(in_features=512, out_features=3)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.lin2 = nn.Linear(in_features=3, out_features=8)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.lin3 = nn.Linear(in_features=8, out_features=3)
        
        
        
    def forward(self, x):
        x = torch.mean(x, dim=1)
        
        x = self.lin1(x)
        x = F.elu(x)
        x = self.drop1(x)
        
        x = self.lin2(x)
        x = F.elu(x)
        x = self.drop2(x)
        
        x = self.lin3(x)
        
        return x
    
    
    
    
class LangIdCNN1(nn.Module):
    def __init__(self):
        super(LangIdCNN1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.3)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.3)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x


    
    
class LangIdRNN1(nn.Module):
    def __init__(self):
        super(LangIdRNN1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.gru = nn.GRU(input_size=1, hidden_size=3, num_layers=1, batch_first=True)
        #self.drop2 = nn.Dropout(p=0.3)
        self.lin2 = nn.Linear(in_features=3, out_features=3)
        
        
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = x.permute(0, 2, 1)
        # output, (h_n, c_n) = self.lstm(x)
        output, h_n = self.gru(x)
        x = output[:, -1, :] # [all batches, last time step, all neurons]
        x = self.lin2(x)
        
        return x
    



# Train Classification Models

In [20]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']    

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)
    train_tags = np.take(audio_tags, train_indices, axis=0)
    train_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        train_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in train_tags]

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    test_tags = np.take(audio_tags, test_indices, axis=0)
    test_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        test_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in test_tags]
        
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name, batch_size)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [21]:
def train_on_fold(model_class, fold_id, feature_name, batch_size, epochs):
    device = torch.device(f"cuda:{GPU_ID}")
    torch.manual_seed(0)
    results = {}
    
    model = model_class().to(device)

    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model_class(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.05)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_n = len(true_train_classes)
        
        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
        train_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes, sample_weight=sw)
            for category, sw in train_bias_category_labels.items()
        }
        


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_n = len(true_test_classes)
        
        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
        test_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes, sample_weight=sw)
            for category, sw in test_bias_category_labels.items()
        }

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'epoch': epoch,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_n': train_n,
            'test_n': test_n,
            
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))
        
    del model
    return results
    

In [22]:
import csv
from pathlib import Path

def save_results(model_name, all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):

                writer.writerow(result_entry['epochs'][epoch])

In [ ]:
model_classes = [
    LangIdCNN1, 
    LangIdRNN1,
    LangIdLogisticReg1, 
    LangIdDNN1
]

all_results = []

for fold_index in cv_folds:
    for model_class in model_classes:
        for feature_name in raw_features:
            print(f"{model_class.__name__} using {feature_name} on fold#{fold_index}")
            resutls = train_on_fold(model_class, fold_index, feature_name, batch_size=100, epochs=1000)
            all_results.append({
                'fold_index': fold_index,
                'feature_name': feature_name,
                'epochs': resutls
            })
            save_results(model_class.__name__, all_results)

LangIdCNN1 using wav2vec_features-c on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 1, 2996]           1,537           1,537
         Dropout-2       [10, 1, 2996]               0               0
       MaxPool1d-3       [10, 1, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              12              12
         Dropout-5       [10, 3, 1496]               0               0
       MaxPool1d-6        [10, 3, 748]               0               0
          Conv1d-7        [10, 3, 746]              30              30
         Dropout-8        [10, 3, 746]               0               0
       MaxPool1d-9        [10, 3, 373]               0               0
         Conv1d-10        [10, 3, 371]              30              30
      MaxPool1d-11        [10, 3, 185]               0               0
Total params: 1,609
Trainable 

Epoch: 840. Train Loss: 0.4497. Test Loss: 1.33. Train Acc: 0.9412. Test Acc:0.5152
Epoch: 850. Train Loss: 0.4469. Test Loss: 1.335. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 860. Train Loss: 0.447. Test Loss: 1.353. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 870. Train Loss: 0.4421. Test Loss: 1.345. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 880. Train Loss: 0.441. Test Loss: 1.337. Train Acc: 0.9412. Test Acc:0.5152
Epoch: 890. Train Loss: 0.4407. Test Loss: 1.362. Train Acc: 0.9608. Test Acc:0.4848
Epoch: 900. Train Loss: 0.4386. Test Loss: 1.336. Train Acc: 0.9412. Test Acc:0.5152
Epoch: 910. Train Loss: 0.4446. Test Loss: 1.387. Train Acc: 0.9412. Test Acc:0.4242
Epoch: 920. Train Loss: 0.4405. Test Loss: 1.345. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 930. Train Loss: 0.4394. Test Loss: 1.373. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 940. Train Loss: 0.4367. Test Loss: 1.374. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 950. Train Loss: 0.4362. Test Loss: 1.363. Train Acc: 0.9608.

Epoch: 680. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 690. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 700. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 710. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 720. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 730. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 740. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 750. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 760. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 770. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 780. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoch: 790. Train Loss: 1.097. Test Loss: 1.103. Train Acc: 0.3529. Test Acc:0.303
Epoc

Epoch: 510. Train Loss: 0.4399. Test Loss: 0.8239. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 520. Train Loss: 0.4372. Test Loss: 0.8274. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 530. Train Loss: 0.4306. Test Loss: 0.8248. Train Acc: 0.9608. Test Acc:0.6061
Epoch: 540. Train Loss: 0.4256. Test Loss: 0.8251. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 550. Train Loss: 0.4229. Test Loss: 0.827. Train Acc: 0.9608. Test Acc:0.6061
Epoch: 560. Train Loss: 0.422. Test Loss: 0.8277. Train Acc: 0.9608. Test Acc:0.6061
Epoch: 570. Train Loss: 0.4159. Test Loss: 0.8291. Train Acc: 0.9608. Test Acc:0.6061
Epoch: 580. Train Loss: 0.4145. Test Loss: 0.8248. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 590. Train Loss: 0.411. Test Loss: 0.828. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 600. Train Loss: 0.407. Test Loss: 0.8245. Train Acc: 0.9608. Test Acc:0.5758
Epoch: 610. Train Loss: 0.4054. Test Loss: 0.8297. Train Acc: 0.9608. Test Acc:0.6061
Epoch: 620. Train Loss: 0.4019. Test Loss: 0.8319. Train Ac

Epoch: 330. Train Loss: 0.638. Test Loss: 0.8029. Train Acc: 0.7059. Test Acc:0.7576
Epoch: 340. Train Loss: 0.6286. Test Loss: 0.7938. Train Acc: 0.6863. Test Acc:0.7576
Epoch: 350. Train Loss: 0.6202. Test Loss: 0.7887. Train Acc: 0.6863. Test Acc:0.7576
Epoch: 360. Train Loss: 0.6159. Test Loss: 0.7804. Train Acc: 0.6863. Test Acc:0.7273
Epoch: 370. Train Loss: 0.6117. Test Loss: 0.7756. Train Acc: 0.6863. Test Acc:0.7576
Epoch: 380. Train Loss: 0.6099. Test Loss: 0.7719. Train Acc: 0.6863. Test Acc:0.7576
Epoch: 390. Train Loss: 0.6058. Test Loss: 0.7669. Train Acc: 0.6863. Test Acc:0.7576
Epoch: 400. Train Loss: 0.6018. Test Loss: 0.7628. Train Acc: 0.6863. Test Acc:0.7576
Epoch: 410. Train Loss: 0.5969. Test Loss: 0.7601. Train Acc: 0.7059. Test Acc:0.7576
Epoch: 420. Train Loss: 0.5914. Test Loss: 0.7559. Train Acc: 0.7255. Test Acc:0.7879
Epoch: 430. Train Loss: 0.5916. Test Loss: 0.7532. Train Acc: 0.7059. Test Acc:0.7576
Epoch: 440. Train Loss: 0.5855. Test Loss: 0.7493. Trai

...

Epoch: 860. Train Loss: 0.3816. Test Loss: 1.268. Train Acc: 1.0. Test Acc:0.3636
Epoch: 870. Train Loss: 0.391. Test Loss: 1.278. Train Acc: 0.9608. Test Acc:0.3636
Epoch: 880. Train Loss: 0.3923. Test Loss: 1.257. Train Acc: 1.0. Test Acc:0.4242
Epoch: 890. Train Loss: 0.3769. Test Loss: 1.27. Train Acc: 1.0. Test Acc:0.4242
Epoch: 900. Train Loss: 0.3759. Test Loss: 1.291. Train Acc: 1.0. Test Acc:0.3939
Epoch: 910. Train Loss: 0.3928. Test Loss: 1.293. Train Acc: 0.9216. Test Acc:0.4242
Epoch: 920. Train Loss: 0.3671. Test Loss: 1.263. Train Acc: 1.0. Test Acc:0.4848
Epoch: 930. Train Loss: 0.3555. Test Loss: 1.312. Train Acc: 1.0. Test Acc:0.3939
Epoch: 940. Train Loss: 0.3698. Test Loss: 1.3. Train Acc: 0.9804. Test Acc:0.4242
Epoch: 950. Train Loss: 0.3841. Test Loss: 1.276. Train Acc: 0.9412. Test Acc:0.4545
Epoch: 960. Train Loss: 0.3594. Test Loss: 1.262. Train Acc: 0.9804. Test Acc:0.4242
Epoch: 970. Train Loss: 0.3617. Test Loss: 1.27. Train Acc: 1.0. Test Acc:0.4242
Epoch:

Epoch: 770. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 780. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 790. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 800. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 810. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 820. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 830. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 840. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 850. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 860. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 870. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 880. Train Loss: 0.9317. Test Loss: 1.013. Train Acc: 0.76

Epoch: 690. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 700. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 710. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 720. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 730. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 740. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 750. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 760. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 770. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 780. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 790. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoch: 800. Train Loss: 1.073. Test Loss: 1.094. Train Acc: 0.451. Test Acc:0.3636
Epoc

Epoch: 610. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 620. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 630. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 640. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 650. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 660. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 670. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 680. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 690. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 700. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 710. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoch: 720. Train Loss: 1.046. Test Loss: 1.086. Train Acc: 0.549. Test Acc:0.3939
Epoc

Epoch: 530. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 540. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 550. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 560. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 570. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 580. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 590. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
Epoch: 600. Train Loss: 1.031. Test Loss: 1.082. Train Acc: 0.6275. Test Acc:0.4848
